In [ ]:
!pip install transformers


In [ ]:
import torch

# Verificar si hay acceso a una GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Crear un tensor en la CPU
tensor_cpu = torch.tensor([1, 2, 3])

# Enviar el tensor a la GPU
tensor_gpu = tensor_cpu.to(device)

# Realizar cálculos en la GPU
resultado_gpu = tensor_gpu * 2

# Obtener el resultado de nuevo en la CPU si es necesario
resultado_cpu = resultado_gpu.to("cpu")

# Imprimir el resultado
print("Resultado en la CPU:", resultado_cpu)


Resultado en la CPU: tensor([2, 4, 6])


In [ ]:
import pandas as pd


In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', nrows=600, usecols=[0, 1, 2, 3], engine='python')

# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_filtered = data.loc[data['label'] != 0]

datatext = data_filtered['text'].astype(str)
print(len(datatext))


223




---


## **MODEL: cardiffnlp/twitter-roberta-base-sentiment**

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Crear el pipeline utilizando el modelo y el tokenizer
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Convertir cada texto en el formato deseado y hacer las predicciones
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))


223


In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_5 = []
x = 0
for i, text in enumerate(formatted_texts):
    try:
        # Tokenizar el texto para obtener los token_type_ids
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Extraer la cadena de texto del tensor de entrada
        text_input = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        # Hacer la predicción con la cadena de texto
        output_5 = sentiment_task(text_input)
        # Obtener la etiqueta de la predicción
        label = output_5[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'LABEL_2':
            predictions_5.append(1)
        elif label == 'LABEL_1':
            predictions_5.append(2)
        else:
            predictions_5.append(3)
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_5.append(3)
    x += 1
    print(f"Texto {x}: {output_5}")

print(predictions_5)


Texto 1: [{'label': 'LABEL_0', 'score': 0.5754660964012146}]
Texto 2: [{'label': 'LABEL_2', 'score': 0.4286562502384186}]
Texto 3: [{'label': 'LABEL_1', 'score': 0.5973053574562073}]
Texto 4: [{'label': 'LABEL_0', 'score': 0.688544750213623}]
Texto 5: [{'label': 'LABEL_1', 'score': 0.5448597073554993}]
Texto 6: [{'label': 'LABEL_1', 'score': 0.4404257535934448}]
Texto 7: [{'label': 'LABEL_1', 'score': 0.6708622574806213}]
Texto 8: [{'label': 'LABEL_1', 'score': 0.44364508986473083}]
Texto 9: [{'label': 'LABEL_1', 'score': 0.4857528507709503}]
Texto 10: [{'label': 'LABEL_0', 'score': 0.5302351713180542}]
Texto 11: [{'label': 'LABEL_1', 'score': 0.44243207573890686}]
Texto 12: [{'label': 'LABEL_0', 'score': 0.7392726540565491}]
Texto 13: [{'label': 'LABEL_0', 'score': 0.7762957215309143}]
Texto 14: [{'label': 'LABEL_2', 'score': 0.7438711524009705}]
Texto 15: [{'label': 'LABEL_0', 'score': 0.7740276455879211}]
Texto 16: [{'label': 'LABEL_1', 'score': 0.49842730164527893}]
Texto 17: [{'la

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data_filtered['label']

# Calcular el conteo de cada clase
class_counts = {label: predictions_5.count(label) for label in set(predictions_5)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_5))


Conteo de cada clase:
Clase 1: 52 instancias
Clase 2: 57 instancias
Clase 3: 114 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.44      0.43      0.43        54
           2       0.67      0.29      0.40       131
           3       0.28      0.84      0.42        38

    accuracy                           0.42       223
   macro avg       0.46      0.52      0.42       223
weighted avg       0.55      0.42      0.41       223





---


## **MODEL: mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis**

https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.

In [ ]:
from transformers import pipeline

# Cargar el modelo de análisis de sentimientos
sentiment_task = pipeline("sentiment-analysis", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", truncation=True, max_length=512)


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_50 = []
x = 0
for i, text in enumerate(formatted_texts):
    try:
        # Hacer la predicción con el texto
        output_50 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_50[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_50.append(1)
        elif label == 'neutral':
            predictions_50.append(2)
        else:
            predictions_50.append(3)
        print(f"Texto {x}: {output_50}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_50.append(3)
    x += 1

print(predictions_50)


Texto 0: [{'label': 'negative', 'score': 0.8217422366142273}]
Texto 1: [{'label': 'neutral', 'score': 0.9998062252998352}]
Texto 2: [{'label': 'neutral', 'score': 0.9998112320899963}]
Texto 3: [{'label': 'neutral', 'score': 0.9995535016059875}]
Texto 4: [{'label': 'neutral', 'score': 0.9998131394386292}]
Texto 5: [{'label': 'neutral', 'score': 0.6483480930328369}]
Texto 6: [{'label': 'neutral', 'score': 0.9991124272346497}]
Texto 7: [{'label': 'positive', 'score': 0.9852876663208008}]
Texto 8: [{'label': 'positive', 'score': 0.7162349820137024}]
Texto 9: [{'label': 'neutral', 'score': 0.999502420425415}]
Texto 10: [{'label': 'negative', 'score': 0.923585057258606}]
Texto 11: [{'label': 'neutral', 'score': 0.9678093194961548}]
Texto 12: [{'label': 'neutral', 'score': 0.9990864992141724}]
Texto 13: [{'label': 'positive', 'score': 0.9992077946662903}]
Texto 14: [{'label': 'neutral', 'score': 0.954571008682251}]
Texto 15: [{'label': 'neutral', 'score': 0.9996026158332825}]
Texto 16: [{'lab

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data_filtered['label']
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_50.count(label) for label in set(predictions_50)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_50))

377    1
378    1
379    1
380    1
381    1
      ..
595    2
596    2
597    2
598    2
599    2
Name: label, Length: 223, dtype: int64
Conteo de cada clase:
Clase 1: 22 instancias
Clase 2: 165 instancias
Clase 3: 36 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.55      0.22      0.32        54
           2       0.67      0.84      0.74       131
           3       0.33      0.32      0.32        38

    accuracy                           0.60       223
   macro avg       0.52      0.46      0.46       223
weighted avg       0.58      0.60      0.57       223

